# cx_oracle examples
## debut des tests

In [1]:
print("debut des tests") 

debut des tests


## initialisation d'oracle client 
il faut telecharger instant client depuis le cite d'oracle avant  , extraire les fichiers dans C:\oracle\instantclient_21_3
avant d'executer cette entrée

In [2]:
import cx_Oracle
cx_Oracle.init_oracle_client(lib_dir=r"C:\oracle\instantclient_21_3")




## connexion a la base de donnée HR 
cette entrée montre comment faire une connecxion a la base de donnée HR 
le nom d'utilisateur de cette BDD c'est 'hr' , le mot de passe aussi .
l'instance de cette BDD s'execute actuelement sur mon localhost donc j'ai preciser 'localhost/xe' comme nom de DNS


In [3]:
import cx_Oracle

con = cx_Oracle.connect(user=r'hr', password='hr', dsn='localhost/xe')
print("la version de la base de donnée:", con.version)
print('connection a la base de données hr faite avec succée')

la version de la base de donnée: 11.2.0.2.0
connection a la base de données hr faite avec succée


## executer une simple requete pour se debuter 
cette entrées montre l'execution d'une reqeutte select appliquée a la table employees du shcema de la bdd hr montrée ci-dessous ,  a travere un curseur

![chema](https://raw.githubusercontent.com/abdelghanimeliani/cx_oracle-examples/master/images/hr.png?token=GHSAT0AAAAAABQADLGYCVNXTZYMTKKN7SVOYVBLF4Q )


la requette est terminée avec succes sauf qu'elle n'affiche rien jusqu'a present 

In [4]:
cursor = con.cursor()
cursor.execute('select * from employees') 

<cx_Oracle.Cursor on <cx_Oracle.Connection to hr@localhost/xe>>

## affichage des lignes une par une

In [5]:
cursor = con.cursor()
cursor.execute('select * from employees') 
res = cursor.fetchall()
for row in res:
    print(row)
print('**********************************')
print('requette select faite avec succes')


(100, 'Steven', 'King', 'SKING', '515.123.4567', datetime.datetime(2003, 6, 17, 0, 0), 'AD_PRES', 24000.0, None, None, 90)
(101, 'Neena', 'Kochhar', 'NKOCHHAR', '515.123.4568', datetime.datetime(2005, 9, 21, 0, 0), 'AD_VP', 17000.0, None, 100, 90)
(102, 'Lex', 'De Haan', 'LDEHAAN', '515.123.4569', datetime.datetime(2001, 1, 13, 0, 0), 'AD_VP', 17000.0, None, 100, 90)
(103, 'Alexander', 'Hunold', 'AHUNOLD', '590.423.4567', datetime.datetime(2006, 1, 3, 0, 0), 'IT_PROG', 9000.0, None, 102, 60)
(104, 'Bruce', 'Ernst', 'BERNST', '590.423.4568', datetime.datetime(2007, 5, 21, 0, 0), 'IT_PROG', 6000.0, None, 103, 60)
(105, 'David', 'Austin', 'DAUSTIN', '590.423.4569', datetime.datetime(2005, 6, 25, 0, 0), 'IT_PROG', 4800.0, None, 103, 60)
(106, 'Valli', 'Pataballa', 'VPATABAL', '590.423.4560', datetime.datetime(2006, 2, 5, 0, 0), 'IT_PROG', 4800.0, None, 103, 60)
(107, 'Diana', 'Lorentz', 'DLORENTZ', '590.423.5567', datetime.datetime(2007, 2, 7, 0, 0), 'IT_PROG', 4200.0, None, 103, 60)
(108,

## fermiture de la connecxion

In [6]:
cursor.close()
con.close()


## Mise en commun des connexions
La mise en commun des connexions est importante pour les performances lorsque des applications multithreads se connectent et se déconnectent fréquemment de la base de données. La mise en commun offre également la meilleure prise en charge des fonctions de haute disponibilité d'Oracle. pour plus de documentation s'il vous plait consultez le rapport attaché

In [8]:
import cx_Oracle
import threading


pool = cx_Oracle.SessionPool(user=r'hr', password='hr', dsn='localhost/xe',
                             min = 2, max = 5, increment = 1, threaded = True,
                             getmode = cx_Oracle.SPOOL_ATTRVAL_WAIT)

def Query():
    con = pool.acquire()
    cur = con.cursor()
    for i in range(4):
        cur.execute("select first_name from employees")
        seqval, = cur.fetchone()
        print("Thread", threading.current_thread().name, "fetched sequence =", seqval)

thread1 = threading.Thread(name='#1', target=Query )
thread1.start()

thread2 = threading.Thread(name='#2', target=Query )
thread2.start()

thread1.join(0.1)
thread2.join()


print("All done!")

Thread #1 fetched sequence = Ellen
Thread #2 fetched sequence = Ellen
Thread #1 fetched sequence = Ellen
Thread #2 fetched sequence = Ellen
Thread #1 fetched sequence = Ellen
Thread #2 fetched sequence = Ellen
Thread #1 fetched sequence = Ellen
Thread #2 fetched sequence = Ellen
All done!


une boucle pour le nombre de threads, chaque itération invoquant la méthode Query() :

In [10]:
import cx_Oracle
import threading


pool = cx_Oracle.SessionPool(user=r'hr', password='hr', dsn='localhost/xe',
                             min = 2, max = 5, increment = 1, threaded = True,
                             getmode = cx_Oracle.SPOOL_ATTRVAL_WAIT)

def Query():
    con = pool.acquire()
    cur = con.cursor()
    for i in range(4):
        cur.execute("select last_name from employees")
        seqval, = cur.fetchone()
        print("Thread", threading.current_thread().name, "fetched sequence =", seqval)

numberOfThreads = 2
threadArray = []

for i in range(numberOfThreads):
    thread = threading.Thread(name = '#' + str(i), target = Query)
    threadArray.append(thread)
    thread.start()

for t in threadArray:
    t.join()

print("All done!")



 

Thread #1 fetched sequence = Abel
Thread #0 fetched sequence = Abel
Thread #1 fetched sequence = Abel
Thread #0 fetched sequence = Abel
Thread #1 fetched sequence = Abel
Thread #0 fetched sequence = Abel
Thread #1 fetched sequence = Abel
Thread #0 fetched sequence = Abel
All done!


## Récupération des données
### Une requête simple
Il existe un certain nombre de fonctions que vous pouvez utiliser pour interroger une base de données Oracle, mais les bases de l'interrogation sont toujours les mêmes :

1. Exécuter l'instruction.
2. Lier les valeurs des données (facultatif).
3. Récupérer les résultats de la base de données.

La méthode cursor() ouvre un curseur pour les instructions à utiliser.

La méthode execute() analyse et exécute l'instruction.

La boucle récupère chaque ligne du curseur et décompose le tuple retourné dans les variables deptno, dname, loc, qui sont ensuite imprimées.

### en utilisant la methode fetchone()

In [11]:
import cx_Oracle

print('dans cet exemple on vas appliquer la methode fetchone() deux fois donc selectionner')
con = cx_Oracle.connect(user=r'hr', password='hr', dsn='localhost/xe',)
cur = con.cursor()

cur.execute("select * from employees")
row = cur.fetchone()
print(row)

row = cur.fetchone()
print(row)

dans cet exemple on vas appliquer la methode fetchone() deux fois donc selectionner
(100, 'Steven', 'King', 'SKING', '515.123.4567', datetime.datetime(2003, 6, 17, 0, 0), 'AD_PRES', 24000.0, None, None, 90)
(101, 'Neena', 'Kochhar', 'NKOCHHAR', '515.123.4568', datetime.datetime(2005, 9, 21, 0, 0), 'AD_VP', 17000.0, None, 100, 90)


### en utilisant la methode fetchone()

In [12]:
import cx_Oracle
print('chercher 3 element de la table employees')

con = cx_Oracle.connect(user=r'hr', password='hr', dsn='localhost/xe',)
cur = con.cursor()

cur.execute("select * from employees")
res = cur.fetchmany(numRows = 3)
print(res)

chercher 3 element de la table employees
[(100, 'Steven', 'King', 'SKING', '515.123.4567', datetime.datetime(2003, 6, 17, 0, 0), 'AD_PRES', 24000.0, None, None, 90), (101, 'Neena', 'Kochhar', 'NKOCHHAR', '515.123.4568', datetime.datetime(2005, 9, 21, 0, 0), 'AD_VP', 17000.0, None, 100, 90), (102, 'Lex', 'De Haan', 'LDEHAAN', '515.123.4569', datetime.datetime(2001, 1, 13, 0, 0), 'AD_VP', 17000.0, None, 100, 90)]


### scrolable cursors
Les curseurs déroulants permettent à l'application de se déplacer vers l'avant ou l'arrière dans les résultats de la requête. Ils peuvent être utilisés pour sauter des lignes ou pour se déplacer vers une ligne particulière.

In [13]:
import cx_Oracle



con = cx_Oracle.connect(user=r'hr', password='hr', dsn='localhost/xe',)
cur = con.cursor(scrollable = True)

cur.execute("select * from employees")

cur.scroll(3, mode = "absolute")  # go to second row
print(cur.fetchone())

cur.scroll(-1)                    # go back one row
print(cur.fetchone())

(102, 'Lex', 'De Haan', 'LDEHAAN', '515.123.4569', datetime.datetime(2001, 1, 13, 0, 0), 'AD_VP', 17000.0, None, 100, 90)
(101, 'Neena', 'Kochhar', 'NKOCHHAR', '515.123.4568', datetime.datetime(2005, 9, 21, 0, 0), 'AD_VP', 17000.0, None, 100, 90)


## Données de liaison

Les variables Bind vous permettent de réexécuter des instructions avec de nouvelles valeurs de données sans avoir à ré-analyser l'instruction. La liaison améliore la réutilisation du code, l'évolutivité des applications et peut réduire le risque d'attaques par injection SQL. L'utilisation des variables de liaison est fortement recommandée.

### Liaison dans SELECT
L'instruction contient un espace réservé à la variable bind ":id". L'instruction est exécutée deux fois avec des valeurs différentes pour la clause WHERE.

In [15]:
import cx_Oracle

con = cx_Oracle.connect(user=r'hr', password='hr', dsn='localhost/xe')
cur = con.cursor()

sql = "select * from employees where employee_id = :id order by employee_id"

cur.execute(sql, id = 100)
res = cur.fetchall()
print(res)

cur.execute(sql, id = 120)
res = cur.fetchall()
print(res)

[(100, 'Steven', 'King', 'SKING', '515.123.4567', datetime.datetime(2003, 6, 17, 0, 0), 'AD_PRES', 24000.0, None, None, 90)]
[(120, 'Matthew', 'Weiss', 'MWEISS', '650.123.1234', datetime.datetime(2004, 7, 18, 0, 0), 'ST_MAN', 8000.0, None, 100, 50)]


### Liaison dans INSERTs

dans cette example on va creer une table (my_tap) avec un simple schema , un id , et une chaine de caractères
Le tableau 'rows' contient les données à insérer.

L'appel executemany() insère toutes les lignes. Cet appel utilise le "array binding", qui est un moyen efficace d'insérer plusieurs enregistrements.

La dernière partie du script interroge les résultats et les affiche sous forme de liste de tuples.

In [27]:
import cx_Oracle

con = cx_Oracle.connect(user=r'hr', password='hr', dsn='localhost/xe')
cur = con.cursor()
sql_delete = "drop table mytab"
sql = "create  table mytab (id number, data varchar2(20), constraint my_pk primary key (id))"
cur.execute(sql_delete)
cur.execute(sql)



rows = [ (1, "First" ), (2, "Second" ),
         (3, "Third" ), (4, "Fourth" ),
         (5, "Fifth" ), (6, "Sixth" ),
         (7, "Seventh" ) ]

cur.executemany("insert into mytab(id, data) values (:1, :2)", rows)

# Now query the results back

cur2 = con.cursor()
cur2.execute('select * from mytab')
res = cur2.fetchall()
print(res)


[(1, 'First'), (2, 'Second'), (3, 'Third'), (4, 'Fourth'), (5, 'Fifth'), (6, 'Sixth'), (7, 'Seventh')]


### Batcherrors
La fonctionnalité Batcherrors permet d'identifier les données invalides tout en autorisant l'insertion de données valides.

Modifiez les valeurs des données dans bind_insert.py et créez une ligne avec une clé dupliquée :

Le nouveau code montre la ligne en double incriminée : "ORA-00001 : unique constraint (PYTHONHOL.MY_PK) violated at row offset 6". Cela indique que la 6ème valeur de données (en partant de 0) avait un problème.

In [28]:
import cx_Oracle

cur = con.cursor()

rows = [ (1, "First" ), (2, "Second" ),
         (3, "Third" ), (4, "Fourth" ),
         (5, "Fifth" ), (6, "Sixth" ),
         (6, "Duplicate" ),
         (7, "Seventh" ) ]

cur.executemany("insert into mytab(id, data) values (:1, :2)", rows, batcherrors = True)

for error in cur.getbatcherrors():
    print("Error", error.message.rstrip(), "at row offset", error.offset)

# Now query the results back

cur2 = con.cursor()
cur2.execute('select * from mytab')
res = cur2.fetchall()
print(res)

Error ORA-00001: unique constraint (HR.MY_PK) violated at row offset 0
Error ORA-00001: unique constraint (HR.MY_PK) violated at row offset 1
Error ORA-00001: unique constraint (HR.MY_PK) violated at row offset 2
Error ORA-00001: unique constraint (HR.MY_PK) violated at row offset 3
Error ORA-00001: unique constraint (HR.MY_PK) violated at row offset 4
Error ORA-00001: unique constraint (HR.MY_PK) violated at row offset 5
Error ORA-00001: unique constraint (HR.MY_PK) violated at row offset 6
Error ORA-00001: unique constraint (HR.MY_PK) violated at row offset 7
[(1, 'First'), (2, 'Second'), (3, 'Third'), (4, 'Fourth'), (5, 'Fifth'), (6, 'Sixth'), (7, 'Seventh')]


## PL/SQL

PL/SQL est l'extension du langage procédural d'Oracle à SQL. Les procédures et fonctions PL/SQL sont stockées et exécutées dans la base de données. L'utilisation de PL/SQL permet à toutes les applications de base de données de réutiliser la logique, quelle que soit la manière dont l'application accède à la base de données. De nombreuses opérations liées aux données peuvent être effectuées en PL/SQL plus rapidement qu'en extrayant les données dans un programme (par exemple, Python) et en les traitant ensuite.

### les fonctios pl/sql
Examinez le fichier plsql_func.sql qui crée une fonction stockée PL/SQL myfunc() pour insérer une ligne dans une nouvelle table nommée ptab et renvoyer le double de la valeur insérée :

|execution de la fonction|
|:------------:|
![schema](https://raw.githubusercontent.com/abdelghanimeliani/cx_oracle-examples/master/images/pl_sql_fuction.PNG?token=GHSAT0AAAAAABQADLGZATYHBTICGSTHGA4YYVBLIVA)|


In [30]:
import cx_Oracle

cur = con.cursor()

res = cur.callfunc('myfunc', int, ('abc', 2))
print("le resultat est : " , res)

le resultat est :  4


Ceci utilise callfunc() pour exécuter la fonction. Le deuxième paramètre est le type de la valeur retournée. Il doit être l'un des types pris en charge par cx_Oracle ou l'une des constantes de type définies par cx_Oracle (comme cx_Oracle.NUMBER). Les deux paramètres de la fonction PL/SQL sont transmis sous forme de tuple, les liant aux arguments des paramètres de la fonction.

### les fonctios pl/sql
Examinez le fichier plsql_proc.sql qui crée une procédure PL/SQL myproc() pour accepter deux paramètres. Le deuxième paramètre contient une valeur de retour OUT.

|execution de la fonction|
|:------------:|
![schema](https://raw.githubusercontent.com/abdelghanimeliani/cx_oracle-examples/master/images/pl_sql_procedure.PNG?token=GHSAT0AAAAAABQADLGYY7SKWZHC3J5SWESSYVBLJAA)|

In [31]:
cur = con.cursor()

myvar = cur.var(int)
cur.callproc('myproc', (123, myvar))
print(myvar.getvalue())

246
